Their paper: https://arxiv.org/pdf/2106.10860.pdf

Their code: https://github.com/dblalock/bolt

Paper says "To assess MADDNESS’s effectiveness, we implemented
both it and existing algorithms in C++ and Python."  

But then on the GitHub page it says "MADDNESS has no Python wrapper yet"...??? 

## First let's try an ordinary matrix multiply

In [1]:
import numpy as np

np.random.seed(0)

n, d, m = 10000, 150, 50 # paper: N >> D ≥ M

a = np.random.rand(n, d)
b = np.random.rand(d, m)
c = a.dot(b)

In [2]:
%%timeit 
c = a.dot(b)

12.4 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Now let's compare the speed of that against MADDNESS:

In [ ]:
# Nope not pip-installable :(
# !pip install git+https://github.com/dblalock/bolt.git

Need [SWIG](http://www.swig.org/exec.html) for compilation later: (and let's just make sure we have clang too)

In [3]:
!apt-get -y install swig clang

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang is already the newest version (1:6.0-41~exp5~ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (940 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpackin

In [4]:
# let's clear out the last time we tried this
! rm -rf bolt
# and clone the code
! git clone https://github.com/dblalock/bolt.git

Cloning into 'bolt'...
remote: Enumerating objects: 7681, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 7681 (delta 8), reused 27 (delta 6), pack-reused 7648
Receiving objects: 100% (7681/7681), 346.48 MiB | 30.18 MiB/s, done.
Resolving deltas: 100% (2237/2237), done.
Checking out files: 100% (5331/5331), done.


As per [@husisy's comment](https://github.com/dblalock/bolt/issues/4#issuecomment-912990699), delete the Mercurial `.hg/` directory for the `eigen` package to avoide "Unterminated string constant" error

In [5]:
! rm -rf bolt/cpp/src/external/eigen/.hg

Now let's try to build:

In [6]:
! cd bolt && python setup.py install 

adding dir to path: cpp/src/quantize
adding dir to path: cpp/src/include
adding dir to path: cpp/src/utils
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
running install
running build_ext
building 'bolt._bolt' extension
swigging python/bolt/native.i to python/bolt/native_wrap.cpp
swig -python -c++ -o python/bolt/native_wrap.cpp python/bolt/native.i
python/bolt/config.i:46: Warning 315: Nothing known about 'Eigen::Dynamic'.
python/bolt/config.i:46: Warning 315: Nothing known about 'Eigen::Dynamic'.
python/bolt/config.i:46: Warning 315: Nothing known about 'Eigen::Dynamic'.
python/bolt/config.i:46: Warning 315: Nothing known about 'Eigen::Dynamic'.
python/bolt/config.i:47: Warning 315: Nothing known about 'Eigen::RowMajor'.
python/bolt/config.i:46: Warning 315: Nothing known about 'Eigen::Dyn

....Will add comment(s) to their GitHub Issue on ["Python Build Problems"](https://github.com/dblalock/bolt/issues/4). 


Update: To get around `unknown file type` error,  more mods by @husisy: Change the `srcFiles +=` line in the `setup.py` file:

In [8]:
setup_str = '''
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
from __future__ import absolute_import, print_function

import io
import os
import sys
from glob import glob
from os.path import basename
from os.path import dirname
from os.path import join
from os.path import splitext

from setuptools import find_packages
from setuptools import setup
from setuptools import Extension


# ================================ C++ extension

import numpy

PROJ_DIR = os.path.dirname(os.path.realpath(__file__))
# CPP_SRC_PATH = join(PROJ_DIR, 'cpp', 'src')
CPP_SRC_PATH = join('cpp', 'src')

# Obtain the numpy include directory.  This logic works across numpy versions.
try:
    numpy_include = numpy.get_include()
except AttributeError:
    numpy_include = numpy.get_numpy_include()

# gather up all the source files
# srcFiles = [join(PROJ_DIR, 'python', 'bolt', 'native.i')]
srcFiles = [join('python', 'bolt', 'native.i')]
includeDirs = [numpy_include]
paths = [CPP_SRC_PATH]
for path in paths:
    srcDir = path
    for root, dirNames, fileNames in os.walk(srcDir):
        for dirName in dirNames:
            absPath = os.path.join(root, dirName)
            print('adding dir to path: %s' % absPath)
            globStr = "%s/*.c*" % absPath
            files = glob(globStr)
            if 'eigen/src' not in absPath:  # just include top level
                includeDirs.append(absPath)
            #srcFiles += files  # husisy mod
            exclude_file_ext = ['.cxxlist','.cu','.cmake','.in','.css','.entry','.evaluator','.expression','.main','.preamble','.traits']
            srcFiles += [x for x in files if not any(x.endswith(y) for y in exclude_file_ext)]

# set the compiler flags so it'll build on different platforms (feel free
# to file a  pull request with a fix if it doesn't work on yours)
# note that -march=native implies -mavx and -mavx2; Bolt requires AVX2
extra_args = ['-std=c++14',
              '-fno-rtti',
              '-march=native',
              '-ffast-math']
if sys.platform == 'darwin':
    extra_args.append('-mmacosx-version-min=10.9')
    os.environ['LDFLAGS'] = '-mmacosx-version-min=10.9 -stdlib=libc++ -framework Accelerate'
    os.environ["CC"] = "g++"  # force compiling c as c++
else:  # based on Issue #4
    extra_args += ['-stdlib=libc++']
    os.environ['CC'] = "clang"
    os.environ['CXX'] = "clang++"
    os.environ['LDFLAGS'] = '-lc++'
# else:
    # os.environ["CC"] = "clang++"  # force compiling c as c++


# inplace extension module
includeDirs += [join(PROJ_DIR, 'python', 'bolt')]  # for swig
nativeExt = Extension("_bolt",  # must match cpp header name with leading _
                      srcFiles,
                      define_macros=[('NDEBUG', '1')],
                      include_dirs=includeDirs,
                      # swig_opts=['-c++', '-modern'],
                      swig_opts=['-c++'],
                      extra_compile_args=extra_args
                      # extra_link_args=['-stdlib=libc++'],
                      )

# ================================ Python modules

glob_str = join('python', 'bolt') + '*.py'
modules = [splitext(basename(path))[0] for path in glob(glob_str)]

# ================================ Call to setup()

setup(
    name='pybolt',
    version='0.1.4',
    license='MPL',
    description='Fast approximate matrix and vector operations',
    author='Davis Blalock',
    author_email='dblalock@mit.edu',
    url='https://github.com/dblalock/bolt',
    download_url='https://github.com/dblalock/bolt/archive/0.1.tar.gz',
    packages=['bolt'],
    package_dir={'bolt': 'python/bolt'},
    py_modules=modules,
    include_package_data=True,
    zip_safe=False,
    classifiers=[
        # complete classifier list:
        # http://pypi.python.org/pypi?%3Aaction=list_classifiers
        'Development Status :: 3 - Alpha',
        'Intended Audience :: Developers',
        'License :: OSI Approved :: Mozilla Public License 2.0 (MPL 2.0)',
        'Operating System :: Unix',
        'Operating System :: POSIX',
        'Programming Language :: C',
        'Programming Language :: C++',
        'Programming Language :: Python',
        'Programming Language :: Python :: 2.7',
        'Programming Language :: Python :: Implementation :: CPython',
        'Topic :: Scientific/Engineering :: Artificial Intelligence',
        'Topic :: Utilities',
    ],
    keywords=[
        'Machine Learning', 'Compression', 'Big Data',
        # eg: 'keyword1', 'keyword2', 'keyword3',
    ],
    install_requires=[
        'numpy',
        'scikit-learn',
        'kmc2'
        # 'sphinx_rtd_theme'  # for docs
    ],
    extras_require={
        # eg:
        #   'rst': ['docutils>=0.11'],
        #   ':python_version=="2.6"': ['argparse'],
    },
    ext_modules=[
        nativeExt
    ],
)

'''

In [9]:
# overwrite setup.py with modification
text_file = open("bolt/setup.py", "w")
n = text_file.write(setup_str)
text_file.close()

In [10]:
#check that the change is there
!cat bolt/setup.py


#!/usr/bin/env python
# -*- encoding: utf-8 -*-
from __future__ import absolute_import, print_function

import io
import os
import sys
from glob import glob
from os.path import basename
from os.path import dirname
from os.path import join
from os.path import splitext

from setuptools import find_packages
from setuptools import setup
from setuptools import Extension


# ================================ C++ extension

import numpy

PROJ_DIR = os.path.dirname(os.path.realpath(__file__))
# CPP_SRC_PATH = join(PROJ_DIR, 'cpp', 'src')
CPP_SRC_PATH = join('cpp', 'src')

# Obtain the numpy include directory.  This logic works across numpy versions.
try:
    numpy_include = numpy.get_include()
except AttributeError:
    numpy_include = numpy.get_numpy_include()

# gather up all the source files
# srcFiles = [join(PROJ_DIR, 'python', 'bolt', 'native.i')]
srcFiles = [join('python', 'bolt', 'native.i')]
includeDirs = [numpy_include]
paths = [CPP_SRC_PATH]
for path in paths:
    srcDir = path
    

Try again to build:

In [11]:
! cd bolt && python setup.py install 

adding dir to path: cpp/src/quantize
adding dir to path: cpp/src/include
adding dir to path: cpp/src/external
adding dir to path: cpp/src/utils
adding dir to path: cpp/src/external/eigen-trimmed
adding dir to path: cpp/src/external/eigen
adding dir to path: cpp/src/external/eigen-trimmed/Eigen
adding dir to path: cpp/src/external/eigen-trimmed/unsupported
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/Householder
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SVD
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SparseLU
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/Geometry
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/QR
adding dir t

Seems that `stdexcept` is a Mac-only file. 

So... seems the bolt devs never intended it to be used outside of Macs.  ...hmmm. :-( 
    
What if we just create a blank `stdexcept` file somewhere?

In [12]:
! sudo echo '' > /content/bolt/cpp/src/include/stdexcept

Rebuild...

In [13]:
! cd bolt && python setup.py install 

adding dir to path: cpp/src/quantize
adding dir to path: cpp/src/include
adding dir to path: cpp/src/external
adding dir to path: cpp/src/utils
adding dir to path: cpp/src/external/eigen-trimmed
adding dir to path: cpp/src/external/eigen
adding dir to path: cpp/src/external/eigen-trimmed/Eigen
adding dir to path: cpp/src/external/eigen-trimmed/unsupported
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/Householder
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SVD
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/SparseLU
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/Geometry
adding dir to path: cpp/src/external/eigen-trimmed/Eigen/src/QR
adding dir t

Uh... `vector` is part of c++.  Why is `clang` running here instead of `clang++`??  ...This `bolt` package is...amazing. ;-) 

In [14]:
!which clang++

/usr/bin/clang++


Alright. Let's just focus on *manually* getting that *one* line to compile:

In [15]:
!cd bolt && clang++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DNDEBUG=1 -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -Icpp/src/utils -Icpp/src/external -Icpp/src/include -Icpp/src/quantize -Icpp/src/external/eigen -Icpp/src/external/eigen-trimmed -Icpp/src/external/eigen/blas -Icpp/src/external/eigen/unsupported -Icpp/src/external/eigen/debug -Icpp/src/external/eigen/demos -Icpp/src/external/eigen/cmake -Icpp/src/external/eigen/doc -Icpp/src/external/eigen/scripts -Icpp/src/external/eigen/bench -Icpp/src/external/eigen/Eigen -Icpp/src/external/eigen/failtest -Icpp/src/external/eigen/blas/fortran -Icpp/src/external/eigen/unsupported/Eigen -Icpp/src/external/eigen/unsupported/Eigen/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11 -Icpp/src/external/eigen/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen/unsupported/Eigen/src/Splines -Icpp/src/external/eigen/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/FFT -Icpp/src/external/eigen/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen/unsupported/Eigen/src/BVH -Icpp/src/external/eigen/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch/CUDA -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen/debug/msvc -Icpp/src/external/eigen/debug/gdb -Icpp/src/external/eigen/demos/mix_eigen_and_c -Icpp/src/external/eigen/demos/opengl -Icpp/src/external/eigen/demos/mandelbrot -Icpp/src/external/eigen/doc/snippets -Icpp/src/external/eigen/doc/special_examples -Icpp/src/external/eigen/doc/examples -Icpp/src/external/eigen/bench/spbench -Icpp/src/external/eigen/bench/btl -Icpp/src/external/eigen/bench/tensors -Icpp/src/external/eigen/bench/perf_monitoring -Icpp/src/external/eigen/bench/btl/generic_bench -Icpp/src/external/eigen/bench/btl/cmake -Icpp/src/external/eigen/bench/btl/libs -Icpp/src/external/eigen/bench/btl/data -Icpp/src/external/eigen/bench/btl/actions -Icpp/src/external/eigen/bench/btl/generic_bench/timers -Icpp/src/external/eigen/bench/btl/generic_bench/utils -Icpp/src/external/eigen/bench/btl/generic_bench/init -Icpp/src/external/eigen/bench/btl/generic_bench/static -Icpp/src/external/eigen/bench/btl/libs/ublas -Icpp/src/external/eigen/bench/btl/libs/blaze -Icpp/src/external/eigen/bench/btl/libs/mtl4 -Icpp/src/external/eigen/bench/btl/libs/tensors -Icpp/src/external/eigen/bench/btl/libs/BLAS -Icpp/src/external/eigen/bench/btl/libs/blitz -Icpp/src/external/eigen/bench/btl/libs/eigen2 -Icpp/src/external/eigen/bench/btl/libs/eigen3 -Icpp/src/external/eigen/bench/btl/libs/STL -Icpp/src/external/eigen/bench/btl/libs/tvmet -Icpp/src/external/eigen/bench/btl/libs/gmm -Icpp/src/external/eigen/bench/perf_monitoring/gemm -Icpp/src/external/eigen/Eigen/src -Icpp/src/external/eigen/Eigen/src/QR -Icpp/src/external/eigen/Eigen/src/SVD -Icpp/src/external/eigen/Eigen/src/CholmodSupport -Icpp/src/external/eigen/Eigen/src/OrderingMethods -Icpp/src/external/eigen/Eigen/src/PaStiXSupport -Icpp/src/external/eigen/Eigen/src/LU -Icpp/src/external/eigen/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen/Eigen/src/plugins -Icpp/src/external/eigen/Eigen/src/Jacobi -Icpp/src/external/eigen/Eigen/src/Core -Icpp/src/external/eigen/Eigen/src/Householder -Icpp/src/external/eigen/Eigen/src/PardisoSupport -Icpp/src/external/eigen/Eigen/src/SparseQR -Icpp/src/external/eigen/Eigen/src/misc -Icpp/src/external/eigen/Eigen/src/MetisSupport -Icpp/src/external/eigen/Eigen/src/UmfPackSupport -Icpp/src/external/eigen/Eigen/src/SuperLUSupport -Icpp/src/external/eigen/Eigen/src/Cholesky -Icpp/src/external/eigen/Eigen/src/SparseCholesky -Icpp/src/external/eigen/Eigen/src/StlSupport -Icpp/src/external/eigen/Eigen/src/SPQRSupport -Icpp/src/external/eigen/Eigen/src/Eigenvalues -Icpp/src/external/eigen/Eigen/src/Geometry -Icpp/src/external/eigen/Eigen/src/SparseCore -Icpp/src/external/eigen/Eigen/src/SparseLU -Icpp/src/external/eigen/Eigen/src/LU/arch -Icpp/src/external/eigen/Eigen/src/Core/arch -Icpp/src/external/eigen/Eigen/src/Core/products -Icpp/src/external/eigen/Eigen/src/Core/functors -Icpp/src/external/eigen/Eigen/src/Core/util -Icpp/src/external/eigen/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX512 -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen/Eigen/src/Core/arch/Default -Icpp/src/external/eigen/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen/Eigen/src/Core/arch/ZVector -Icpp/src/external/eigen/Eigen/src/Geometry/arch -Icpp/src/external/eigen-trimmed/unsupported -Icpp/src/external/eigen-trimmed/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test -Icpp/src/external/eigen-trimmed/unsupported/doc -Icpp/src/external/eigen-trimmed/unsupported/bench -Icpp/src/external/eigen-trimmed/unsupported/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test/mpreal -Icpp/src/external/eigen-trimmed/unsupported/doc/snippets -Icpp/src/external/eigen-trimmed/unsupported/doc/examples -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11 -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Splines -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/FFT -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/BVH -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch/GPU -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen-trimmed/Eigen/src -Icpp/src/external/eigen-trimmed/Eigen/src/QR -Icpp/src/external/eigen-trimmed/Eigen/src/SVD -Icpp/src/external/eigen-trimmed/Eigen/src/CholmodSupport -Icpp/src/external/eigen-trimmed/Eigen/src/OrderingMethods -Icpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport -Icpp/src/external/eigen-trimmed/Eigen/src/LU -Icpp/src/external/eigen-trimmed/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen-trimmed/Eigen/src/plugins -Icpp/src/external/eigen-trimmed/Eigen/src/Jacobi -Icpp/src/external/eigen-trimmed/Eigen/src/Core -Icpp/src/external/eigen-trimmed/Eigen/src/Householder -Icpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SparseQR -Icpp/src/external/eigen-trimmed/Eigen/src/misc -Icpp/src/external/eigen-trimmed/Eigen/src/MetisSupport -Icpp/src/external/eigen-trimmed/Eigen/src/UmfPackSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Cholesky -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCholesky -Icpp/src/external/eigen-trimmed/Eigen/src/StlSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SPQRSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCore -Icpp/src/external/eigen-trimmed/Eigen/src/SparseLU -Icpp/src/external/eigen-trimmed/Eigen/src/LU/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/products -Icpp/src/external/eigen-trimmed/Eigen/src/Core/functors -Icpp/src/external/eigen-trimmed/Eigen/src/Core/util -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/Default -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry/arch -I/content/bolt/python/bolt -I/usr/include/python3.7m -c python/bolt/native_wrap.cpp -o build/temp.linux-x86_64-3.7/python/bolt/native_wrap.o -std=c++14 -fno-rtti -march=native -ffast-math -stdlib=libc++

python/bolt/native_wrap.cpp:3127:10: fatal error: 'vector' file not found
#include <vector>
         ^~~~~~~~
1 error generated.


In [16]:
!ls /usr/include/c++/7.5.0/vector

/usr/include/c++/7.5.0/vector


Let's add `-I/usr/include/c++/7.5.0 -I/usr/include/x86_64-linux-gnu/c++/7.5.0`

In [17]:
!cd bolt && clang++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DNDEBUG=1 -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -Icpp/src/utils -Icpp/src/external -Icpp/src/include -Icpp/src/quantize -Icpp/src/external/eigen -Icpp/src/external/eigen-trimmed -Icpp/src/external/eigen/blas -Icpp/src/external/eigen/unsupported -Icpp/src/external/eigen/debug -Icpp/src/external/eigen/demos -Icpp/src/external/eigen/cmake -Icpp/src/external/eigen/doc -Icpp/src/external/eigen/scripts -Icpp/src/external/eigen/bench -Icpp/src/external/eigen/Eigen -Icpp/src/external/eigen/failtest -Icpp/src/external/eigen/blas/fortran -Icpp/src/external/eigen/unsupported/Eigen -Icpp/src/external/eigen/unsupported/Eigen/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11 -Icpp/src/external/eigen/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen/unsupported/Eigen/src/Splines -Icpp/src/external/eigen/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/FFT -Icpp/src/external/eigen/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen/unsupported/Eigen/src/BVH -Icpp/src/external/eigen/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch/CUDA -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen/debug/msvc -Icpp/src/external/eigen/debug/gdb -Icpp/src/external/eigen/demos/mix_eigen_and_c -Icpp/src/external/eigen/demos/opengl -Icpp/src/external/eigen/demos/mandelbrot -Icpp/src/external/eigen/doc/snippets -Icpp/src/external/eigen/doc/special_examples -Icpp/src/external/eigen/doc/examples -Icpp/src/external/eigen/bench/spbench -Icpp/src/external/eigen/bench/btl -Icpp/src/external/eigen/bench/tensors -Icpp/src/external/eigen/bench/perf_monitoring -Icpp/src/external/eigen/bench/btl/generic_bench -Icpp/src/external/eigen/bench/btl/cmake -Icpp/src/external/eigen/bench/btl/libs -Icpp/src/external/eigen/bench/btl/data -Icpp/src/external/eigen/bench/btl/actions -Icpp/src/external/eigen/bench/btl/generic_bench/timers -Icpp/src/external/eigen/bench/btl/generic_bench/utils -Icpp/src/external/eigen/bench/btl/generic_bench/init -Icpp/src/external/eigen/bench/btl/generic_bench/static -Icpp/src/external/eigen/bench/btl/libs/ublas -Icpp/src/external/eigen/bench/btl/libs/blaze -Icpp/src/external/eigen/bench/btl/libs/mtl4 -Icpp/src/external/eigen/bench/btl/libs/tensors -Icpp/src/external/eigen/bench/btl/libs/BLAS -Icpp/src/external/eigen/bench/btl/libs/blitz -Icpp/src/external/eigen/bench/btl/libs/eigen2 -Icpp/src/external/eigen/bench/btl/libs/eigen3 -Icpp/src/external/eigen/bench/btl/libs/STL -Icpp/src/external/eigen/bench/btl/libs/tvmet -Icpp/src/external/eigen/bench/btl/libs/gmm -Icpp/src/external/eigen/bench/perf_monitoring/gemm -Icpp/src/external/eigen/Eigen/src -Icpp/src/external/eigen/Eigen/src/QR -Icpp/src/external/eigen/Eigen/src/SVD -Icpp/src/external/eigen/Eigen/src/CholmodSupport -Icpp/src/external/eigen/Eigen/src/OrderingMethods -Icpp/src/external/eigen/Eigen/src/PaStiXSupport -Icpp/src/external/eigen/Eigen/src/LU -Icpp/src/external/eigen/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen/Eigen/src/plugins -Icpp/src/external/eigen/Eigen/src/Jacobi -Icpp/src/external/eigen/Eigen/src/Core -Icpp/src/external/eigen/Eigen/src/Householder -Icpp/src/external/eigen/Eigen/src/PardisoSupport -Icpp/src/external/eigen/Eigen/src/SparseQR -Icpp/src/external/eigen/Eigen/src/misc -Icpp/src/external/eigen/Eigen/src/MetisSupport -Icpp/src/external/eigen/Eigen/src/UmfPackSupport -Icpp/src/external/eigen/Eigen/src/SuperLUSupport -Icpp/src/external/eigen/Eigen/src/Cholesky -Icpp/src/external/eigen/Eigen/src/SparseCholesky -Icpp/src/external/eigen/Eigen/src/StlSupport -Icpp/src/external/eigen/Eigen/src/SPQRSupport -Icpp/src/external/eigen/Eigen/src/Eigenvalues -Icpp/src/external/eigen/Eigen/src/Geometry -Icpp/src/external/eigen/Eigen/src/SparseCore -Icpp/src/external/eigen/Eigen/src/SparseLU -Icpp/src/external/eigen/Eigen/src/LU/arch -Icpp/src/external/eigen/Eigen/src/Core/arch -Icpp/src/external/eigen/Eigen/src/Core/products -Icpp/src/external/eigen/Eigen/src/Core/functors -Icpp/src/external/eigen/Eigen/src/Core/util -Icpp/src/external/eigen/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX512 -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen/Eigen/src/Core/arch/Default -Icpp/src/external/eigen/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen/Eigen/src/Core/arch/ZVector -Icpp/src/external/eigen/Eigen/src/Geometry/arch -Icpp/src/external/eigen-trimmed/unsupported -Icpp/src/external/eigen-trimmed/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test -Icpp/src/external/eigen-trimmed/unsupported/doc -Icpp/src/external/eigen-trimmed/unsupported/bench -Icpp/src/external/eigen-trimmed/unsupported/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test/mpreal -Icpp/src/external/eigen-trimmed/unsupported/doc/snippets -Icpp/src/external/eigen-trimmed/unsupported/doc/examples -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11 -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Splines -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/FFT -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/BVH -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch/GPU -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen-trimmed/Eigen/src -Icpp/src/external/eigen-trimmed/Eigen/src/QR -Icpp/src/external/eigen-trimmed/Eigen/src/SVD -Icpp/src/external/eigen-trimmed/Eigen/src/CholmodSupport -Icpp/src/external/eigen-trimmed/Eigen/src/OrderingMethods -Icpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport -Icpp/src/external/eigen-trimmed/Eigen/src/LU -Icpp/src/external/eigen-trimmed/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen-trimmed/Eigen/src/plugins -Icpp/src/external/eigen-trimmed/Eigen/src/Jacobi -Icpp/src/external/eigen-trimmed/Eigen/src/Core -Icpp/src/external/eigen-trimmed/Eigen/src/Householder -Icpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SparseQR -Icpp/src/external/eigen-trimmed/Eigen/src/misc -Icpp/src/external/eigen-trimmed/Eigen/src/MetisSupport -Icpp/src/external/eigen-trimmed/Eigen/src/UmfPackSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Cholesky -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCholesky -Icpp/src/external/eigen-trimmed/Eigen/src/StlSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SPQRSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCore -Icpp/src/external/eigen-trimmed/Eigen/src/SparseLU -Icpp/src/external/eigen-trimmed/Eigen/src/LU/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/products -Icpp/src/external/eigen-trimmed/Eigen/src/Core/functors -Icpp/src/external/eigen-trimmed/Eigen/src/Core/util -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AltiVec -I/usr/include/c++/7.5.0 -I/usr/include/x86_64-linux-gnu/c++/7.5.0 -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/Default -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry/arch -I/content/bolt/python/bolt -I/usr/include/python3.7m -c python/bolt/native_wrap.cpp -o build/temp.linux-x86_64-3.7/python/bolt/native_wrap.o -std=c++14 -fno-rtti -march=native -ffast-math -stdlib=libc++

In file included from python/bolt/native_wrap.cpp:3129:
In file included from /content/bolt/python/bolt/../../cpp/src/include/public.hpp:19:
In file included from cpp/src/external/eigen/Eigen/Core:96:
In file included from /usr/include/c++/7.5.0/complex:45:
In file included from /usr/include/c++/7.5.0/sstream:38:
In file included from /usr/include/c++/7.5.0/istream:38:
In file included from /usr/include/c++/7.5.0/ios:42:
In file included from /usr/include/c++/7.5.0/bits/ios_base.h:46:
/usr/include/c++/7.5.0/system_error:94:13: error: unknown type name '__cow_string'
    virtual __cow_string
            ^
/usr/include/c++/7.5.0/system_error:341:36: error: expected class name
  class system_error : public std::runtime_error
                                   ^
/usr/include/c++/7.5.0/system_error:348:7: error: member initializer 'runtime_error' does not name a non-static data member or base class
    : runtime_error(__ec.message()), _M_code(__ec) { }
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/u

In [18]:
!cd bolt && clang++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DNDEBUG=1 -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -Icpp/src/utils -Icpp/src/external -Icpp/src/include -Icpp/src/quantize -Icpp/src/external/eigen -Icpp/src/external/eigen-trimmed -Icpp/src/external/eigen/blas -Icpp/src/external/eigen/unsupported -Icpp/src/external/eigen/debug -Icpp/src/external/eigen/demos -Icpp/src/external/eigen/cmake -Icpp/src/external/eigen/doc -Icpp/src/external/eigen/scripts -Icpp/src/external/eigen/bench -Icpp/src/external/eigen/Eigen -Icpp/src/external/eigen/failtest -Icpp/src/external/eigen/blas/fortran -Icpp/src/external/eigen/unsupported/Eigen -Icpp/src/external/eigen/unsupported/Eigen/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11 -Icpp/src/external/eigen/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen/unsupported/Eigen/src/Splines -Icpp/src/external/eigen/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/FFT -Icpp/src/external/eigen/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen/unsupported/Eigen/src/BVH -Icpp/src/external/eigen/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch/CUDA -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen/debug/msvc -Icpp/src/external/eigen/debug/gdb -Icpp/src/external/eigen/demos/mix_eigen_and_c -Icpp/src/external/eigen/demos/opengl -Icpp/src/external/eigen/demos/mandelbrot -Icpp/src/external/eigen/doc/snippets -Icpp/src/external/eigen/doc/special_examples -Icpp/src/external/eigen/doc/examples -Icpp/src/external/eigen/bench/spbench -Icpp/src/external/eigen/bench/btl -Icpp/src/external/eigen/bench/tensors -Icpp/src/external/eigen/bench/perf_monitoring -Icpp/src/external/eigen/bench/btl/generic_bench -Icpp/src/external/eigen/bench/btl/cmake -Icpp/src/external/eigen/bench/btl/libs -Icpp/src/external/eigen/bench/btl/data -Icpp/src/external/eigen/bench/btl/actions -Icpp/src/external/eigen/bench/btl/generic_bench/timers -Icpp/src/external/eigen/bench/btl/generic_bench/utils -Icpp/src/external/eigen/bench/btl/generic_bench/init -Icpp/src/external/eigen/bench/btl/generic_bench/static -Icpp/src/external/eigen/bench/btl/libs/ublas -Icpp/src/external/eigen/bench/btl/libs/blaze -Icpp/src/external/eigen/bench/btl/libs/mtl4 -Icpp/src/external/eigen/bench/btl/libs/tensors -Icpp/src/external/eigen/bench/btl/libs/BLAS -Icpp/src/external/eigen/bench/btl/libs/blitz -Icpp/src/external/eigen/bench/btl/libs/eigen2 -Icpp/src/external/eigen/bench/btl/libs/eigen3 -Icpp/src/external/eigen/bench/btl/libs/STL -Icpp/src/external/eigen/bench/btl/libs/tvmet -Icpp/src/external/eigen/bench/btl/libs/gmm -Icpp/src/external/eigen/bench/perf_monitoring/gemm -Icpp/src/external/eigen/Eigen/src -Icpp/src/external/eigen/Eigen/src/QR -Icpp/src/external/eigen/Eigen/src/SVD -Icpp/src/external/eigen/Eigen/src/CholmodSupport -Icpp/src/external/eigen/Eigen/src/OrderingMethods -Icpp/src/external/eigen/Eigen/src/PaStiXSupport -Icpp/src/external/eigen/Eigen/src/LU -Icpp/src/external/eigen/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen/Eigen/src/plugins -Icpp/src/external/eigen/Eigen/src/Jacobi -Icpp/src/external/eigen/Eigen/src/Core -Icpp/src/external/eigen/Eigen/src/Householder -Icpp/src/external/eigen/Eigen/src/PardisoSupport -Icpp/src/external/eigen/Eigen/src/SparseQR -Icpp/src/external/eigen/Eigen/src/misc -Icpp/src/external/eigen/Eigen/src/MetisSupport -Icpp/src/external/eigen/Eigen/src/UmfPackSupport -Icpp/src/external/eigen/Eigen/src/SuperLUSupport -Icpp/src/external/eigen/Eigen/src/Cholesky -Icpp/src/external/eigen/Eigen/src/SparseCholesky -Icpp/src/external/eigen/Eigen/src/StlSupport -Icpp/src/external/eigen/Eigen/src/SPQRSupport -Icpp/src/external/eigen/Eigen/src/Eigenvalues -Icpp/src/external/eigen/Eigen/src/Geometry -Icpp/src/external/eigen/Eigen/src/SparseCore -Icpp/src/external/eigen/Eigen/src/SparseLU -Icpp/src/external/eigen/Eigen/src/LU/arch -Icpp/src/external/eigen/Eigen/src/Core/arch -Icpp/src/external/eigen/Eigen/src/Core/products -Icpp/src/external/eigen/Eigen/src/Core/functors -Icpp/src/external/eigen/Eigen/src/Core/util -Icpp/src/external/eigen/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX512 -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen/Eigen/src/Core/arch/Default -Icpp/src/external/eigen/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen/Eigen/src/Core/arch/ZVector -Icpp/src/external/eigen/Eigen/src/Geometry/arch -Icpp/src/external/eigen-trimmed/unsupported -Icpp/src/external/eigen-trimmed/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test -Icpp/src/external/eigen-trimmed/unsupported/doc -Icpp/src/external/eigen-trimmed/unsupported/bench -Icpp/src/external/eigen-trimmed/unsupported/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test/mpreal -Icpp/src/external/eigen-trimmed/unsupported/doc/snippets -Icpp/src/external/eigen-trimmed/unsupported/doc/examples -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11 -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Splines -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/FFT -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/BVH -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch/GPU -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen-trimmed/Eigen/src -Icpp/src/external/eigen-trimmed/Eigen/src/QR -Icpp/src/external/eigen-trimmed/Eigen/src/SVD -Icpp/src/external/eigen-trimmed/Eigen/src/CholmodSupport -Icpp/src/external/eigen-trimmed/Eigen/src/OrderingMethods -Icpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport -Icpp/src/external/eigen-trimmed/Eigen/src/LU -Icpp/src/external/eigen-trimmed/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen-trimmed/Eigen/src/plugins -Icpp/src/external/eigen-trimmed/Eigen/src/Jacobi -Icpp/src/external/eigen-trimmed/Eigen/src/Core -Icpp/src/external/eigen-trimmed/Eigen/src/Householder -Icpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SparseQR -Icpp/src/external/eigen-trimmed/Eigen/src/misc -Icpp/src/external/eigen-trimmed/Eigen/src/MetisSupport -Icpp/src/external/eigen-trimmed/Eigen/src/UmfPackSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Cholesky -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCholesky -Icpp/src/external/eigen-trimmed/Eigen/src/StlSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SPQRSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCore -Icpp/src/external/eigen-trimmed/Eigen/src/SparseLU -Icpp/src/external/eigen-trimmed/Eigen/src/LU/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/products -Icpp/src/external/eigen-trimmed/Eigen/src/Core/functors -Icpp/src/external/eigen-trimmed/Eigen/src/Core/util -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AltiVec -I/usr/include/c++/7.5.0 -I/usr/include/x86_64-linux-gnu/c++/7.5.0 -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/Default -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry/arch -I/content/bolt/python/bolt -I/usr/include/python3.7m -c python/bolt/native_wrap.cpp -o build/temp.linux-x86_64-3.7/python/bolt/native_wrap.o -std=c++14 -fno-rtti -march=native -ffast-math -stdlib=libc++ -DENABLE_PRECOMPILED_HEADERS=OFF

In file included from python/bolt/native_wrap.cpp:3129:
In file included from /content/bolt/python/bolt/../../cpp/src/include/public.hpp:19:
In file included from cpp/src/external/eigen/Eigen/Core:96:
In file included from /usr/include/c++/7.5.0/complex:45:
In file included from /usr/include/c++/7.5.0/sstream:38:
In file included from /usr/include/c++/7.5.0/istream:38:
In file included from /usr/include/c++/7.5.0/ios:42:
In file included from /usr/include/c++/7.5.0/bits/ios_base.h:46:
/usr/include/c++/7.5.0/system_error:94:13: error: unknown type name '__cow_string'
    virtual __cow_string
            ^
/usr/include/c++/7.5.0/system_error:341:36: error: expected class name
  class system_error : public std::runtime_error
                                   ^
/usr/include/c++/7.5.0/system_error:348:7: error: member initializer 'runtime_error' does not name a non-static data member or base class
    : runtime_error(__ec.message()), _M_code(__ec) { }
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/u

Ok some of those are no doubt due to my setting the error file to blank.

But also, WTH is it lower-case "egien/Core" instead of "Eigen/Core"??  Other packages (PyTorch, Tensorflow,...) have the latter. And there's an `Eigen/src/Core` here: 

In [19]:
!ls bolt/cpp/src/external/eigen-trimmed/Eigen/src/Core

arch		    Dot.h		  Redux.h
ArrayBase.h	    EigenBase.h		  Ref.h
Array.h		    ForceAlignedAccess.h  Replicate.h
ArrayWrapper.h	    functors		  ReturnByValue.h
AssignEvaluator.h   Fuzzy.h		  Reverse.h
Assign.h	    GeneralProduct.h	  Select.h
Assign_MKL.h	    GenericPacketMath.h   SelfAdjointView.h
BandMatrix.h	    GlobalFunctions.h	  SelfCwiseBinaryOp.h
Block.h		    Inverse.h		  Solve.h
BooleanRedux.h	    IO.h		  SolverBase.h
CMakeLists.txt	    MapBase.h		  SolveTriangular.h
CommaInitializer.h  Map.h		  SpecialFunctions.h
CoreEvaluators.h    MathFunctions.h	  StableNorm.h
CoreIterators.h     MatrixBase.h	  Stride.h
CwiseBinaryOp.h     Matrix.h		  Swap.h
CwiseNullaryOp.h    NestByValue.h	  Transpose.h
CwiseUnaryOp.h	    NoAlias.h		  Transpositions.h
CwiseUnaryView.h    NumTraits.h		  TriangularMatrix.h
DenseBase.h	    PermutationMatrix.h   util
DenseCoeffsBase.h   PlainObjectBase.h	  VectorBlock.h
DenseStorage.h	    ProductEvaluators.h   VectorwiseOp.h
Diagonal.h	    Product.h		  Visitor.

Do we need to install full `eigen3`??

In [20]:
!sudo apt-get install libeigen3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  libeigen3-doc libmrpt-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 810 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeigen3-dev all 3.3.4-4 [810 kB]
Fetched 810 kB in 1s (798 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)


In [21]:
!ls /usr/include/eigen3/eigen

ls: cannot access '/usr/include/eigen3/eigen': No such file or directory


In [22]:
!cd bolt && clang++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DNDEBUG=1 -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -Icpp/src/utils -Icpp/src/external -Icpp/src/include -Icpp/src/quantize -Icpp/src/external/eigen -Icpp/src/external/eigen-trimmed -Icpp/src/external/eigen/blas -Icpp/src/external/eigen/unsupported -Icpp/src/external/eigen/debug -Icpp/src/external/eigen/demos -Icpp/src/external/eigen/cmake -Icpp/src/external/eigen/doc -Icpp/src/external/eigen/scripts -Icpp/src/external/eigen/bench -Icpp/src/external/eigen/Eigen -Icpp/src/external/eigen/failtest -Icpp/src/external/eigen/blas/fortran -Icpp/src/external/eigen/unsupported/Eigen -Icpp/src/external/eigen/unsupported/Eigen/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11 -Icpp/src/external/eigen/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen/unsupported/Eigen/src/Splines -Icpp/src/external/eigen/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/FFT -Icpp/src/external/eigen/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen/unsupported/Eigen/src/BVH -Icpp/src/external/eigen/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch/CUDA -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen/debug/msvc -Icpp/src/external/eigen/debug/gdb -Icpp/src/external/eigen/demos/mix_eigen_and_c -Icpp/src/external/eigen/demos/opengl -Icpp/src/external/eigen/demos/mandelbrot -Icpp/src/external/eigen/doc/snippets -Icpp/src/external/eigen/doc/special_examples -Icpp/src/external/eigen/doc/examples -Icpp/src/external/eigen/bench/spbench -Icpp/src/external/eigen/bench/btl -Icpp/src/external/eigen/bench/tensors -Icpp/src/external/eigen/bench/perf_monitoring -Icpp/src/external/eigen/bench/btl/generic_bench -Icpp/src/external/eigen/bench/btl/cmake -Icpp/src/external/eigen/bench/btl/libs -Icpp/src/external/eigen/bench/btl/data -Icpp/src/external/eigen/bench/btl/actions -Icpp/src/external/eigen/bench/btl/generic_bench/timers -Icpp/src/external/eigen/bench/btl/generic_bench/utils -Icpp/src/external/eigen/bench/btl/generic_bench/init -Icpp/src/external/eigen/bench/btl/generic_bench/static -Icpp/src/external/eigen/bench/btl/libs/ublas -Icpp/src/external/eigen/bench/btl/libs/blaze -Icpp/src/external/eigen/bench/btl/libs/mtl4 -Icpp/src/external/eigen/bench/btl/libs/tensors -Icpp/src/external/eigen/bench/btl/libs/BLAS -Icpp/src/external/eigen/bench/btl/libs/blitz -Icpp/src/external/eigen/bench/btl/libs/eigen2 -Icpp/src/external/eigen/bench/btl/libs/eigen3 -Icpp/src/external/eigen/bench/btl/libs/STL -Icpp/src/external/eigen/bench/btl/libs/tvmet -Icpp/src/external/eigen/bench/btl/libs/gmm -Icpp/src/external/eigen/bench/perf_monitoring/gemm -Icpp/src/external/eigen/Eigen/src -Icpp/src/external/eigen/Eigen/src/QR -Icpp/src/external/eigen/Eigen/src/SVD -Icpp/src/external/eigen/Eigen/src/CholmodSupport -Icpp/src/external/eigen/Eigen/src/OrderingMethods -Icpp/src/external/eigen/Eigen/src/PaStiXSupport -Icpp/src/external/eigen/Eigen/src/LU -Icpp/src/external/eigen/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen/Eigen/src/plugins -Icpp/src/external/eigen/Eigen/src/Jacobi -Icpp/src/external/eigen/Eigen/src/Core -Icpp/src/external/eigen/Eigen/src/Householder -Icpp/src/external/eigen/Eigen/src/PardisoSupport -Icpp/src/external/eigen/Eigen/src/SparseQR -Icpp/src/external/eigen/Eigen/src/misc -Icpp/src/external/eigen/Eigen/src/MetisSupport -Icpp/src/external/eigen/Eigen/src/UmfPackSupport -Icpp/src/external/eigen/Eigen/src/SuperLUSupport -Icpp/src/external/eigen/Eigen/src/Cholesky -Icpp/src/external/eigen/Eigen/src/SparseCholesky -Icpp/src/external/eigen/Eigen/src/StlSupport -Icpp/src/external/eigen/Eigen/src/SPQRSupport -Icpp/src/external/eigen/Eigen/src/Eigenvalues -Icpp/src/external/eigen/Eigen/src/Geometry -Icpp/src/external/eigen/Eigen/src/SparseCore -Icpp/src/external/eigen/Eigen/src/SparseLU -Icpp/src/external/eigen/Eigen/src/LU/arch -Icpp/src/external/eigen/Eigen/src/Core/arch -Icpp/src/external/eigen/Eigen/src/Core/products -Icpp/src/external/eigen/Eigen/src/Core/functors -Icpp/src/external/eigen/Eigen/src/Core/util -Icpp/src/external/eigen/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX512 -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen/Eigen/src/Core/arch/Default -Icpp/src/external/eigen/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen/Eigen/src/Core/arch/ZVector -Icpp/src/external/eigen/Eigen/src/Geometry/arch -Icpp/src/external/eigen-trimmed/unsupported -Icpp/src/external/eigen-trimmed/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test -Icpp/src/external/eigen-trimmed/unsupported/doc -Icpp/src/external/eigen-trimmed/unsupported/bench -Icpp/src/external/eigen-trimmed/unsupported/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test/mpreal -Icpp/src/external/eigen-trimmed/unsupported/doc/snippets -Icpp/src/external/eigen-trimmed/unsupported/doc/examples -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11 -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Splines -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/FFT -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/BVH -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch/GPU -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen-trimmed/Eigen/src -Icpp/src/external/eigen-trimmed/Eigen/src/QR -Icpp/src/external/eigen-trimmed/Eigen/src/SVD -Icpp/src/external/eigen-trimmed/Eigen/src/CholmodSupport -Icpp/src/external/eigen-trimmed/Eigen/src/OrderingMethods -Icpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport -Icpp/src/external/eigen-trimmed/Eigen/src/LU -Icpp/src/external/eigen-trimmed/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen-trimmed/Eigen/src/plugins -Icpp/src/external/eigen-trimmed/Eigen/src/Jacobi -Icpp/src/external/eigen-trimmed/Eigen/src/Core -Icpp/src/external/eigen-trimmed/Eigen/src/Householder -Icpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SparseQR -Icpp/src/external/eigen-trimmed/Eigen/src/misc -Icpp/src/external/eigen-trimmed/Eigen/src/MetisSupport -Icpp/src/external/eigen-trimmed/Eigen/src/UmfPackSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Cholesky -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCholesky -Icpp/src/external/eigen-trimmed/Eigen/src/StlSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SPQRSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCore -Icpp/src/external/eigen-trimmed/Eigen/src/SparseLU -Icpp/src/external/eigen-trimmed/Eigen/src/LU/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/products -Icpp/src/external/eigen-trimmed/Eigen/src/Core/functors -Icpp/src/external/eigen-trimmed/Eigen/src/Core/util -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AltiVec -I/usr/include/c++/7.5.0 -I/usr/include/x86_64-linux-gnu/c++/7.5.0 -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/Default -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry/arch -I/content/bolt/python/bolt -I/usr/include/python3.7m -c python/bolt/native_wrap.cpp -o build/temp.linux-x86_64-3.7/python/bolt/native_wrap.o -std=c++14 -fno-rtti -march=native -ffast-math -stdlib=libc++ -DENABLE_PRECOMPILED_HEADERS=OFF -I/usr/include/eigen3/

In file included from python/bolt/native_wrap.cpp:3129:
In file included from /content/bolt/python/bolt/../../cpp/src/include/public.hpp:19:
In file included from cpp/src/external/eigen/Eigen/Core:96:
In file included from /usr/include/c++/7.5.0/complex:45:
In file included from /usr/include/c++/7.5.0/sstream:38:
In file included from /usr/include/c++/7.5.0/istream:38:
In file included from /usr/include/c++/7.5.0/ios:42:
In file included from /usr/include/c++/7.5.0/bits/ios_base.h:46:
/usr/include/c++/7.5.0/system_error:94:13: error: unknown type name '__cow_string'
    virtual __cow_string
            ^
/usr/include/c++/7.5.0/system_error:341:36: error: expected class name
  class system_error : public std::runtime_error
                                   ^
/usr/include/c++/7.5.0/system_error:348:7: error: member initializer 'runtime_error' does not name a non-static data member or base class
    : runtime_error(__ec.message()), _M_code(__ec) { }
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/u

Uh... can we just....?

In [23]:
!sudo ln -s /usr/include/eigen3/Eigen/ /usr/include/eigen3/eigen 

In [24]:
!cd bolt && clang++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DNDEBUG=1 -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -Icpp/src/utils -Icpp/src/external -Icpp/src/include -Icpp/src/quantize -Icpp/src/external/eigen -Icpp/src/external/eigen-trimmed -Icpp/src/external/eigen/blas -Icpp/src/external/eigen/unsupported -Icpp/src/external/eigen/debug -Icpp/src/external/eigen/demos -Icpp/src/external/eigen/cmake -Icpp/src/external/eigen/doc -Icpp/src/external/eigen/scripts -Icpp/src/external/eigen/bench -Icpp/src/external/eigen/Eigen -Icpp/src/external/eigen/failtest -Icpp/src/external/eigen/blas/fortran -Icpp/src/external/eigen/unsupported/Eigen -Icpp/src/external/eigen/unsupported/Eigen/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11 -Icpp/src/external/eigen/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen/unsupported/Eigen/src/Splines -Icpp/src/external/eigen/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen/unsupported/Eigen/src/FFT -Icpp/src/external/eigen/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen/unsupported/Eigen/src/BVH -Icpp/src/external/eigen/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen/unsupported/Eigen/src/SpecialFunctions/arch/CUDA -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen/debug/msvc -Icpp/src/external/eigen/debug/gdb -Icpp/src/external/eigen/demos/mix_eigen_and_c -Icpp/src/external/eigen/demos/opengl -Icpp/src/external/eigen/demos/mandelbrot -Icpp/src/external/eigen/doc/snippets -Icpp/src/external/eigen/doc/special_examples -Icpp/src/external/eigen/doc/examples -Icpp/src/external/eigen/bench/spbench -Icpp/src/external/eigen/bench/btl -Icpp/src/external/eigen/bench/tensors -Icpp/src/external/eigen/bench/perf_monitoring -Icpp/src/external/eigen/bench/btl/generic_bench -Icpp/src/external/eigen/bench/btl/cmake -Icpp/src/external/eigen/bench/btl/libs -Icpp/src/external/eigen/bench/btl/data -Icpp/src/external/eigen/bench/btl/actions -Icpp/src/external/eigen/bench/btl/generic_bench/timers -Icpp/src/external/eigen/bench/btl/generic_bench/utils -Icpp/src/external/eigen/bench/btl/generic_bench/init -Icpp/src/external/eigen/bench/btl/generic_bench/static -Icpp/src/external/eigen/bench/btl/libs/ublas -Icpp/src/external/eigen/bench/btl/libs/blaze -Icpp/src/external/eigen/bench/btl/libs/mtl4 -Icpp/src/external/eigen/bench/btl/libs/tensors -Icpp/src/external/eigen/bench/btl/libs/BLAS -Icpp/src/external/eigen/bench/btl/libs/blitz -Icpp/src/external/eigen/bench/btl/libs/eigen2 -Icpp/src/external/eigen/bench/btl/libs/eigen3 -Icpp/src/external/eigen/bench/btl/libs/STL -Icpp/src/external/eigen/bench/btl/libs/tvmet -Icpp/src/external/eigen/bench/btl/libs/gmm -Icpp/src/external/eigen/bench/perf_monitoring/gemm -Icpp/src/external/eigen/Eigen/src -Icpp/src/external/eigen/Eigen/src/QR -Icpp/src/external/eigen/Eigen/src/SVD -Icpp/src/external/eigen/Eigen/src/CholmodSupport -Icpp/src/external/eigen/Eigen/src/OrderingMethods -Icpp/src/external/eigen/Eigen/src/PaStiXSupport -Icpp/src/external/eigen/Eigen/src/LU -Icpp/src/external/eigen/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen/Eigen/src/plugins -Icpp/src/external/eigen/Eigen/src/Jacobi -Icpp/src/external/eigen/Eigen/src/Core -Icpp/src/external/eigen/Eigen/src/Householder -Icpp/src/external/eigen/Eigen/src/PardisoSupport -Icpp/src/external/eigen/Eigen/src/SparseQR -Icpp/src/external/eigen/Eigen/src/misc -Icpp/src/external/eigen/Eigen/src/MetisSupport -Icpp/src/external/eigen/Eigen/src/UmfPackSupport -Icpp/src/external/eigen/Eigen/src/SuperLUSupport -Icpp/src/external/eigen/Eigen/src/Cholesky -Icpp/src/external/eigen/Eigen/src/SparseCholesky -Icpp/src/external/eigen/Eigen/src/StlSupport -Icpp/src/external/eigen/Eigen/src/SPQRSupport -Icpp/src/external/eigen/Eigen/src/Eigenvalues -Icpp/src/external/eigen/Eigen/src/Geometry -Icpp/src/external/eigen/Eigen/src/SparseCore -Icpp/src/external/eigen/Eigen/src/SparseLU -Icpp/src/external/eigen/Eigen/src/LU/arch -Icpp/src/external/eigen/Eigen/src/Core/arch -Icpp/src/external/eigen/Eigen/src/Core/products -Icpp/src/external/eigen/Eigen/src/Core/functors -Icpp/src/external/eigen/Eigen/src/Core/util -Icpp/src/external/eigen/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen/Eigen/src/Core/arch/AltiVec -Icpp/src/external/eigen/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX512 -Icpp/src/external/eigen/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen/Eigen/src/Core/arch/Default -Icpp/src/external/eigen/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen/Eigen/src/Core/arch/ZVector -Icpp/src/external/eigen/Eigen/src/Geometry/arch -Icpp/src/external/eigen-trimmed/unsupported -Icpp/src/external/eigen-trimmed/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test -Icpp/src/external/eigen-trimmed/unsupported/doc -Icpp/src/external/eigen-trimmed/unsupported/bench -Icpp/src/external/eigen-trimmed/unsupported/Eigen -Icpp/src/external/eigen-trimmed/unsupported/test/mpreal -Icpp/src/external/eigen-trimmed/unsupported/doc/snippets -Icpp/src/external/eigen-trimmed/unsupported/doc/examples -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11 -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SparseExtra -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Splines -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NumericalDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/AutoDiff -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/NonLinearOptimization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Skyline -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/IterativeSolvers -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/KroneckerProduct -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/LevenbergMarquardt -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MatrixFunctions -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/FFT -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Polynomials -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/BVH -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/MoreVectorization -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/EulerAngles -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch -Icpp/src/external/eigen-trimmed/unsupported/Eigen/src/SpecialFunctions/arch/GPU -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/Tensor -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/ThreadPool -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/util -Icpp/src/external/eigen-trimmed/unsupported/Eigen/CXX11/src/TensorSymmetry/util -Icpp/src/external/eigen-trimmed/Eigen/src -Icpp/src/external/eigen-trimmed/Eigen/src/QR -Icpp/src/external/eigen-trimmed/Eigen/src/SVD -Icpp/src/external/eigen-trimmed/Eigen/src/CholmodSupport -Icpp/src/external/eigen-trimmed/Eigen/src/OrderingMethods -Icpp/src/external/eigen-trimmed/Eigen/src/PaStiXSupport -Icpp/src/external/eigen-trimmed/Eigen/src/LU -Icpp/src/external/eigen-trimmed/Eigen/src/IterativeLinearSolvers -Icpp/src/external/eigen-trimmed/Eigen/src/plugins -Icpp/src/external/eigen-trimmed/Eigen/src/Jacobi -Icpp/src/external/eigen-trimmed/Eigen/src/Core -Icpp/src/external/eigen-trimmed/Eigen/src/Householder -Icpp/src/external/eigen-trimmed/Eigen/src/PardisoSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SparseQR -Icpp/src/external/eigen-trimmed/Eigen/src/misc -Icpp/src/external/eigen-trimmed/Eigen/src/MetisSupport -Icpp/src/external/eigen-trimmed/Eigen/src/UmfPackSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SuperLUSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Cholesky -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCholesky -Icpp/src/external/eigen-trimmed/Eigen/src/StlSupport -Icpp/src/external/eigen-trimmed/Eigen/src/SPQRSupport -Icpp/src/external/eigen-trimmed/Eigen/src/Eigenvalues -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry -Icpp/src/external/eigen-trimmed/Eigen/src/SparseCore -Icpp/src/external/eigen-trimmed/Eigen/src/SparseLU -Icpp/src/external/eigen-trimmed/Eigen/src/LU/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch -Icpp/src/external/eigen-trimmed/Eigen/src/Core/products -Icpp/src/external/eigen-trimmed/Eigen/src/Core/functors -Icpp/src/external/eigen-trimmed/Eigen/src/Core/util -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/SSE -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AltiVec -I/usr/include/c++/7.5.0 -I/usr/include/x86_64-linux-gnu/c++/7.5.0 -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/CUDA -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/AVX -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/Default -Icpp/src/external/eigen-trimmed/Eigen/src/Core/arch/NEON -Icpp/src/external/eigen-trimmed/Eigen/src/Geometry/arch -I/content/bolt/python/bolt -I/usr/include/python3.7m -c python/bolt/native_wrap.cpp -o build/temp.linux-x86_64-3.7/python/bolt/native_wrap.o -std=c++14 -fno-rtti -march=native -ffast-math -stdlib=libc++ -DENABLE_PRECOMPILED_HEADERS=OFF -I/usr/include/eigen3/

In file included from python/bolt/native_wrap.cpp:3129:
In file included from /content/bolt/python/bolt/../../cpp/src/include/public.hpp:19:
In file included from cpp/src/external/eigen/Eigen/Core:96:
In file included from /usr/include/c++/7.5.0/complex:45:
In file included from /usr/include/c++/7.5.0/sstream:38:
In file included from /usr/include/c++/7.5.0/istream:38:
In file included from /usr/include/c++/7.5.0/ios:42:
In file included from /usr/include/c++/7.5.0/bits/ios_base.h:46:
/usr/include/c++/7.5.0/system_error:94:13: error: unknown type name '__cow_string'
    virtual __cow_string
            ^
/usr/include/c++/7.5.0/system_error:341:36: error: expected class name
  class system_error : public std::runtime_error
                                   ^
/usr/include/c++/7.5.0/system_error:348:7: error: member initializer 'runtime_error' does not name a non-static data member or base class
    : runtime_error(__ec.message()), _M_code(__ec) { }
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/u

CF this thread: https://github.com/opencv/opencv/issues/14868. Recommends changing that include line to read: 

`#include <eigen3/Eigen/Core>`.


...but on Colab it's hard to edit files, so let's do violence to the filesystem instead:

In [25]:
!ln -s bolt/cpp/src/external/eigen/Eigen/ bolt/cpp/src/external/eigen 

ln: failed to create symbolic link 'bolt/cpp/src/external/eigen/Eigen': File exists


??? how can Colab's filesystem be case-insensitive?

In [26]:
!ls bolt/cpp/src/external/eigen

bench		COPYING.MINPACK    Eigen
blas		COPYING.MPL2	   failtest
cmake		COPYING.README	   INSTALL
CMakeLists.txt	CTestConfig.cmake  README.md
COPYING.BSD	debug		   scripts
COPYING.GPL	demos		   signature_of_eigen3_matrix_library
COPYING.LGPL	doc		   unsupported


...so just the upper-case version of `Eigen` is there. No lower-case.  

:shrug:

In [27]:
!ls bolt/cpp/src/external/eigen-trimmed/Eigen

Cholesky	Householder		QR		 src
CholmodSupport	IterativeLinearSolvers	QtAlignedMalloc  StdDeque
CMakeLists.txt	Jacobi			Sparse		 StdList
Core		LU			SparseCholesky	 StdVector
Dense		MetisSupport		SparseCore	 SuperLUSupport
Eigen		OrderingMethods		SparseLU	 SVD
Eigenvalues	PardisoSupport		SparseQR	 UmfPackSupport
Geometry	PaStiXSupport		SPQRSupport


Enough of this.  I've spent hours on this. I'm out.